In [18]:
# !pip install --upgrade pip         # pip 19.0 or higher is required for TF 2
# !pip install --upgrade setuptools  # Otherwise you'll get annoying warnings about bad installs
!yes | pip uninstall protobuf
!yes | pip uninstall tensorflow
!yes | pip uninstall tensorflow-gpu
!yes | pip install --upgrade --force-reinstall tensorflow-gpu

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Found existing installation: protobuf 3.11.4
Uninstalling protobuf-3.11.4:
  Would remove:
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p27/lib/python2.7/site-packages/protobuf-3.11.4-py2.7.egg-info
Proceed (y/n)?   Successfully uninstalled protobuf-3.11.4
yes: standard output: Broken pipe
yes: write error
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/develop

  Created wheel for wrapt: filename=wrapt-1.12.1-cp27-cp27mu-linux_x86_64.whl size=63754 sha256=523ae599c41bbd34f12681d912ae00782a52cce945a791874061479dc0a80842
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5b/d8/8e/81a83cb5321b940a954996f5b57fddc8976e712b3ac3a1a54b
  Created wheel for keras-applications: filename=Keras_Applications-1.0.8-py2-none-any.whl size=50701 sha256=7bbde717f7383ca1d3d2d98c167ddb46fc06228e37fd4bebec7af16e0a1ecd9e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/71/a0/64/e2e0c93740e0460f4b7f036141b7c73b5e44ff38f690ddff9f
  Created wheel for opt-einsum: filename=opt_einsum-2.3.2-py2-none-any.whl size=49881 sha256=85ad7434b317d11e6edfc067a675ca865179431cfe1e5943e54dc428824d4fe2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ef/c4/c2/d0b07dd2a54f4d583a5de0e6ce5eb7a1832961b9a10d1ec953
  Created wheel for functools32: filename=functools32-3.2.3.post2-py2-none-any.whl size=14556 sha256=25223c4d3c35d898db3be65bf2cf2957f11fe15ed73c90b7852e61ef70

In [24]:
!export CUDA_VISIBLE_DEVICES=0,1

In [25]:
# !pip install --user --upgrade tensorflow-gpu

In [26]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name=u'/physical_device:GPU:0', device_type=u'GPU')]

In [19]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) #Make full screen width

In [2]:
_ = !pip install --ignore-installed --upgrade tensorflow==2.4.0
_ = !pip install wandb
# _ = !pip install dirsync
import wandb
wandb.init()

  Using cached tensorflow-2.4.0-cp36-cp36m-manylinux2010_x86_64.whl (394.7 MB)
  Using cached absl_py-0.11.0-py3-none-any.whl (127 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.32.0-cp36-cp36m-manylinux2014_x86_64.whl (3.8 MB)
  Using cached h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl (2.9 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-3.14.0-cp36-cp36m-manylinux1_x86_64.whl (1.0 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached tensorboard-2.4.0-py3-none-any.whl (10.6 MB)
  Using cached google_auth-1.24.0-py2.py3-none-any.whl (114 kB)
  Using cached cachetools-4.2.0-

In [3]:
import re; import sys; import importlib; import tqdm; import os; import time; import glob
import numpy as np; import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [4]:
#This is to import local modules
sys.path.append('/home/ec2-user/SageMaker/PythonFiles')
import FunctionsTF as F
importlib.reload(F) #Incase need to reload module after change
import Metrics as M
importlib.reload(M) #Incase need to reload module after change

<module 'Metrics' from '/home/ec2-user/SageMaker/PythonFiles/Metrics.py'>

In [5]:
homeDirectory = '/home/ec2-user/SageMaker/'

model_name = r'20201216_460k_Param_LSTM_Skip_resBlock_311Epoch.h5'
model_save_location = homeDirectory + r'Models/' + model_name
output_model_save_location = homeDirectory + r'Models/' + r'20201216_460k_Param_LSTM_Skip_resBlock.h5'

historyPath = homeDirectory + r'LossCurves/' + r'20201216History.csv'

bucket_name = 'hilcorp-l48operations-plunger-lift-main'

In [6]:
#This creates a new hitory df if one does not already exist
if not os.path.isfile(historyPath):
    print('Creating new history DF at {}'.format(time.localtime()))
    pd.DataFrame(columns = ['loss', 'MCF_metric', 'plunger_speed_metric', 'val_loss', 'val_MCF_metric', 'val_plunger_speed_metric']).to_csv(historyPath, index = False)

In [7]:
model = load_model(model_save_location, compile = False, custom_objects = {'LeakyReLU' : LeakyReLU()})
model.summary()# tf.keras.utils.plot_model(model,show_shapes=True)

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, 79)]   0                                            
__________________________________________________________________________________________________
batch_normalization_107 (BatchN (None, None, 79)     316         input_4[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, None, 32)     14336       batch_normalization_107[0][0]    
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, None, 32)     14336       batch_normalization_107[0][0]    
_______________________________________________________________________________________

In [8]:
list_of_files = glob.glob('/home/ec2-user/SageMaker/TFRecordFiles/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime) #This gets the most recently uploaded TF Record File
lTFRecordFiles = [latest_file]
print(lTFRecordFiles)

['/home/ec2-user/SageMaker/TFRecordFiles/DatasetOneExamplePerWellWithUWI-5138-Records.tfrecords']


In [9]:
validation_split = 0.1
batch_size = 2
num_parallel_calls = 8
buffer_size = 8

In [10]:
def count_data_items(filenames):
    'Counts the records in each file name'
    n = [int(re.compile(r"-([0-9]*)-Records\.").search(filename).group(1)) for filename in filenames]
#     print(n)
    return np.sum(n)

In [11]:
num_examples = count_data_items(lTFRecordFiles)

In [12]:
numTrainWells = int(np.floor(num_examples*(1.-validation_split)))
numValidWells = num_examples - numTrainWells
print(f"Number of training wells: {numTrainWells}, Validation wells: {numValidWells} of total wells {num_examples}")

Number of training wells: 4624, Validation wells: 514 of total wells 5138


In [13]:
raw_dataset = tf.data.TFRecordDataset(lTFRecordFiles)
# allWellDs = allWellDs.map(lambda x, y: (x[:100,:],y[:100,:]))#This is just for testing purposes to trim X for shorter computation

trainDs = raw_dataset.skip(numValidWells)
trainDs = trainDs.map(F.parse_raw_examples_UWI, num_parallel_calls=num_parallel_calls)
trainDs = trainDs.map(lambda x, y, UWIs: (x,y))#This is to remove the UWI which is not useful in trianing

trainDs = trainDs.map(lambda x, y: (tf.reverse(x, axis = [0]),tf.reverse(y, axis = [0])))#This is to have leading instead of trailing 0s. Reverse the time direction
trainDs = trainDs.padded_batch(batch_size, padded_shapes=([None,79],[None,2]))# Add the 0s behind the example
trainDs = trainDs.map(lambda x, y: (tf.reverse(x, axis = [1]),tf.reverse(y, axis = [1]))) #Reverse the time to the correct direction
trainDs = trainDs.prefetch(buffer_size)
# trainDs = trainDs.cache(r'./')

validDs = raw_dataset.take(numValidWells)
validDs = validDs.map(F.parse_raw_examples_UWI, num_parallel_calls=num_parallel_calls)
validDs = validDs.map(lambda x, y, UWIs: (x,y))#This is to remove the UWI which is not useful in trianing
validDs = validDs.map(lambda x, y: (tf.reverse(x, axis = [0]),tf.reverse(y, axis = [0])))
validDs = validDs.padded_batch(batch_size, padded_shapes=([None,79],[None,2]))
validDs = validDs.map(lambda x, y: (tf.reverse(x, axis = [1]),tf.reverse(y, axis = [1])))
validDs = validDs.prefetch(buffer_size)

In [14]:
for x in tqdm.tqdm_notebook(trainDs.take(5)): pass

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [15]:
for x in tqdm.tqdm_notebook(validDs.take(5)): pass

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [16]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Mon Jan 11 20:54:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
steps_per_epoch = int(np.ceil(numTrainWells/batch_size))

In [18]:
class EpochLogger(tf.keras.callbacks.Callback):
    def __init__(self,historyPath):
        self.historyPath = historyPath
    def on_epoch_end(self,epoch,logs=None):#This saves the loss data
        lossDf = pd.DataFrame(logs, index = [0]) #Turns logs into dataframe
        lossDf.to_csv(self.historyPath, mode = 'a', header = False, index = False) #Appends to existing csv file

model_checkpoint = ModelCheckpoint(model_save_location, 
                                   monitor = 'loss', 
                                   save_best_only=False, 
                                   save_weights_only = False,
                                   verbose=1)

terminateOnNaN = keras.callbacks.TerminateOnNaN()
log_results = EpochLogger(historyPath)

In [19]:
epochs = 1000

optimizer = Adam(lr = 1e-3)
model.compile(loss=M.custom_loss, optimizer=optimizer, metrics = [M.MCF_metric, M.plunger_speed_metric])

model.fit(x = trainDs.repeat(epochs),
          validation_data=validDs,
          epochs = epochs,
          steps_per_epoch = steps_per_epoch,
          use_multiprocessing=False,
          callbacks = [
            log_results,
            model_checkpoint,
            terminateOnNaN
            ]
          )

Epoch 1/1000
   5/2312 [..............................] - ETA: 34:03:00 - loss: 19638.5912 - MCF_metric: 1599.3271 - plunger_speed_metric: 37593.5543

KeyboardInterrupt: 